# **1 - Data Preparation**
---

In [1]:
# Import required libraries.
import yaml
import pandas as pd
import joblib

from sklearn.model_selection import train_test_split

## 0. Load Config
---

In [2]:
# Function to load configuration file.
def load_config(config_dir: str) -> dict:
    """
    Load the configuration file.
    
    Parameters:
    ----------
    config_dir : str
        The configuration file location.
        
    Returns:
    -------
    config : dict
        The loaded configuration file.
    """
    # Try to load the yaml file.
    try:
        with open(config_dir, 'r') as file:
            config = yaml.safe_load(file)
    except FileNotFoundError as err:
        raise RuntimeError("Configuration file not found in the path.")
    
    # Return the loaded config file.
    return config

In [3]:
# Load the configuration file.
CONFIG_DIR = "../config/config.yaml"
config = load_config(CONFIG_DIR)

## 1. Load Data
---

In [4]:
# Function to load data.
def load_data(data_dir: str) -> pd.DataFrame:
    """
    Load the csv data.
    
    Parameters:
    ----------
    data_dir : str
        The location of dataset.
        
    Returns:
    -------
    data : pd.DataFrame
        The loaded data.
    """
    # Load the csv data.
    data = pd.read_csv(data_dir)
    
    # Print the data shape.
    print(f"Raw data shape: {data.shape}")
    
    # Return the loaded data as Pandas DataFrame.
    return data

In [5]:
# Load the csv data and check the top 5 data.
DATA_DIR = f"../{config["path_dataset"]}"

data = load_data(DATA_DIR)
data.head()

Raw data shape: (20000, 19)


,Patient_ID,Age,Gender,Tumor_Type,Tumor_Size,Location,Histology,Stage,Symptom_1,Symptom_2,Symptom_3,Radiation_Treatment,Surgery_Performed,Chemotherapy,Survival_Rate,Tumor_Growth_Rate,Family_History,MRI_Result,Follow_Up_Required
0,1,73,Male,Malignant,5.375612,Temporal,Astrocytoma,III,Vision Issues,Seizures,Seizures,No,No,No,51.312579,0.111876,No,Positive,Yes
1,2,26,Male,Benign,4.847098,Parietal,Glioblastoma,II,Headache,Headache,Nausea,Yes,Yes,Yes,46.373273,2.165736,Yes,Positive,Yes
2,3,31,Male,Benign,5.588391,Parietal,Meningioma,I,Vision Issues,Headache,Seizures,No,No,No,47.072221,1.884228,No,Negative,No
3,4,29,Male,Malignant,1.436600,Temporal,Medulloblastoma,IV,Vision Issues,Seizures,Headache,Yes,No,Yes,51.853634,1.283342,Yes,Negative,No
4,5,54,Female,Benign,2.417506,Parietal,Glioblastoma,I,Headache,Headache,Seizures,No,No,Yes,54.708987,2.069477,No,Positive,Yes


## 2. Split Data
---
1. `X`-`y` split
2. Train-Test Split

In [6]:
# Function to split input (X) and output (y).
def split_input_output(data: pd.DataFrame, target_col: str):
    """
    Split the input and output features.
    
    Parameters:
    ----------
    data : pd.DataFrame
        The loaded dataset.
        
    target_col : str
        The name of target column.
        
    Returns:
    -------
    X : pd.DataFrame
        The input data.
        
    y : pd.Series
        The output data.
    """
    # Input features.
    X = data.drop(columns = target_col,
                  axis = 1)
    
    # Output features.
    y = data[target_col]
    
    # Print the data shape.
    print(f"Original data shape : {data.shape}")
    print(f"X data shape        : {X.shape}")
    print(f"y data shape        : {y.shape}")
    
    # Return the input and output data.
    return X, y

In [7]:
# Split input-output.
TARGET_COL = config['label']

X, y = split_input_output(data = data,
                          target_col = TARGET_COL)

Original data shape : (20000, 19)
X data shape        : (20000, 18)
y data shape        : (20000,)


In [8]:
# Function to split train and test data.
def split_train_test(X: pd.DataFrame, y: pd.Series, test_size: float, random_state: int = None):
    """
    Stratify split train test.
        Stratify: target proportion before and after split will be same.
        
    Parameters:
    ----------
    X : pd.DataFrame
        The input data.
    
    y : pd.Series
        The output data.
        
    test_size : float
        The proportion of test data.
        
    random_state : int, default = None
        State of randomness, for reproducibility.
        
    Returns:
    -------
    X_train : pd.DataFrame
        The input data train.
        
    X_test : pd.DataFrame
        The input data test.
        
    y_train : pd.Series
        The output data train.
        
    y_test : pd.Series
        The output data test.
    """
    # Split the data into train and test set.
    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size = test_size,
        stratify = y,
        random_state = random_state
    )
    
    # Print the data shape.
    print(f"X_train shape: {X_train.shape}")
    print(f"y_train shape: {y_train.shape}")
    print(f"X_test shape : {X_test.shape}")
    print(f"y_test shape : {y_test.shape}")
    
    # Return the splitted data.
    return X_train, X_test, y_train, y_test

In [9]:
# Split train-test from X and y.
X_train, X_test, y_train, y_test = split_train_test(
    X = X,
    y = y,
    test_size = 0.2,
    random_state = 123
)

X_train shape: (16000, 18)
y_train shape: (16000,)
X_test shape : (4000, 18)
y_test shape : (4000,)


## 3. Serialize Data
---

In [10]:
# Function to serialize data.
def pickle_dump(data: pd.DataFrame, path: str):
    """
    Serialize data into pickle file.
    
    Parameters:
    ----------
    data : pd.DataFrame
        The loaded data.
        
    path : str
        The destination path.
        
    Returns:
    -------
    None
    """
    # Serialize the data into the path.
    joblib.dump(data, path)
    print("Data serialized.")

In [11]:
# Serialize the train data.
pickle_dump(X_train, f"../{config["path_train_set"][0]}")
pickle_dump(y_train, f"../{config["path_train_set"][1]}")

# Serialize the test data.
pickle_dump(X_test, f"../{config["path_test_set"][0]}")
pickle_dump(y_test, f"../{config["path_test_set"][1]}")

Data serialized.
Data serialized.
Data serialized.
Data serialized.
